In [42]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.neural_network import MLPRegressor

param_file = 'Snow17_NN_params.csv'
SWE_sim_file = 'Snow17_NN_SWE.csv'
forcing_file = 'Snow17_NN_forcing.csv'


## 2. Split Training and Testing Dataset

In [43]:
df_params = pd.read_csv(param_file)
df_swe_sim = pd.read_csv(SWE_sim_file)
df_forcing = pd.read_csv(forcing_file)

Nt = df_swe_sim.shape[0]
Nreps = df_params.shape[0]

print('Nt = '+str(Nt))
print('Nreps = '+str(Nreps))


Nt = 7305
Nreps = 1000


## 3. Assemble Training and Test Datasets Into Correct Format

In [44]:
# Process our "Obervations" we want the neural net to predict. In this case SWE
# Get the SWE ensemble and reshape it into a 1-D array, column-wise 
SWE_ens = df_swe_sim.iloc[:,2:].values
SWE_ens_1D = SWE_ens.T.reshape(Nt*Nreps,1)

print(SWE_ens_1D.shape)

(7305000, 1)


In [48]:
# Process our forcings, making a copy for each ensemble member
tair = df_forcing['tair'].values.reshape((Nt,1))
tair_ens = np.tile(tair,(Nreps,1))
print('Shape of tair_ens = '+str(tair_ens.shape))

pcp = df_forcing['pcp'].values.reshape((Nt,1))
pcp_ens = np.tile(pcp,(Nreps,1))
print('Shape of pcp_ens = '+str(pcp_ens.shape))

Shape of tair_ens = (7305000, 1)
Shape of pcp_ens = (7305000, 1)


In [54]:
# Process our parameters making a copy for each date
Dd = df_params['Dd_ens'].values.reshape((1,Nreps))
Dd_ens = np.tile(Dd,(Nt,1)).T.reshape((Nt*Nreps,1))

Tt = df_params['Tt_ens'].values.reshape((1,Nreps))
Tt_ens = np.tile(Tt,(Nt,1)).T.reshape((Nt*Nreps,1))

print('Shape of Dd_ens = '+str(Dd_ens.shape))
print('Shape of Tt_ens = '+str(Tt_ens.shape))

Shape of Dd_ens = (7305000, 1)
Shape of Tt_ens = (7305000, 1)


In [58]:
# Assemble predictor array
SWE_predictors = np.concatenate((pcp_ens,tair_ens,Dd_ens,Tt_ens),axis=1)

print(SWE_predictors)


[[ 0.         10.55555556  4.48386247 -0.30981407]
 [ 0.         10.55555556  4.48386247 -0.30981407]
 [ 0.          8.33333333  4.48386247 -0.30981407]
 ...
 [ 0.          4.44444444  1.89457969  1.84999293]
 [ 0.          9.44444444  1.89457969  1.84999293]
 [ 0.         10.55555556  1.89457969  1.84999293]]


## 4. Train the Neural Network Model